In [ ]:
import os
import json
import shutil

from library_dicom.dicom_processor.model.Series import Series
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.tools.folders import get_series_path, write_json_file, remove_bi_file
from library_dicom.dicom_processor.tools.series import get_series_object


In [ ]:
series_paths = get_series_path("/media/salim/DD 2To/AHL2011")
export_folder = '/media/salim/DD 2To/Json Ahl'

In [ ]:
not_image_folder=[]
non_valid_series=[]
non_attenuation_corrected=[]
empty_folders = []
secondary_series = []
localizer_series = []

for serie_path in series_paths:
    if(len(os.listdir(serie_path)) == 0):
        empty_folders.append(serie_path)
        continue
    try:    
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        json_string = json.dumps(dicomsInfo)
        remove_bi_file(serie_path)
        if( dicom_serie.is_image_modality() == False):
            not_image_folder.append(serie_path)
        elif (dicom_serie.is_series_valid() == False):
            non_valid_series.append(serie_path)
        elif(type(dicom_serie) == SeriesPT and dicom_serie.is_corrected_attenuation() == False):
            non_attenuation_corrected.append(serie_path)
        elif(dicom_serie.is_localizer_series() == True):
            localizer_series.append(serie_path)
        else:
            write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], json_string)
    except Exception as err:
        print(err)
        print(serie_path)
        print(dicomsInfo)
    
    
    

In [ ]:
#print(not_image_folder)
#print(non_valid_series)
#print(non_attenuation_corrected)
print(empty_folders)
#print(secondary_series)
#print(localizer_series)

In [ ]:
for localizer_serie in localizer_series:
    shutil.rmtree(localizer_serie)

for not_image in not_image_folder:
    shutil.rmtree(not_image)

for non_attenuation in non_attenuation_corrected:
    shutil.rmtree(non_attenuation)

In [ ]:
for empty_folder in empty_folders:
    shutil.rmtree(empty_folder)

In [ ]:
write_json_file("/media/salim/DD 2To/Erreur GAINED", "wrong Series Gained", json.dumps(non_valid_series))